In [1]:
import pandas as pd
import cvxpy as cp
import timeit
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [2]:
def Cost(C_i, i=0.04/12, periods=20*12): # i = interest rate per month, periods = payback periods (months), can change timescale to not be months
    fv = C_i*((1+i)**periods) #future value calculation
    cost = fv*(i/(((1+i)**periods) - 1))*(1/(1+i)) # monthly payment
    return cost

def GridCost(cg, mon, t, cost, c_fix):
    if (mon=="june"):
        df = pd.read_excel ('houston pricing.xlsx', sheet_name='Jun')
        if (cost=="fixed"):
            for n in range (0, t):
                cg += [c_fix] 
        elif (cost=="wholesale"):
            for n in range (0, t):
                cg += [df.at[n, "Settlement Point Price"]]
        else:
            print('invalid cost input. must be "fixed" or "wholesale"')
            return 0
    elif (mon=="december"):
        if (cost=="fixed"):
            for n in range (0, t):
                cg += [c_fix] 
        elif (cost=="wholesale"):
            df = pd.read_excel ('houston pricing.xlsx', sheet_name='Dec')
            for n in range (0, t):
                cg += [df.at[n, "Settlement Point Price"]]
        else:
            print('invalid cost input. must be "fixed" or "wholesale"')
            return 0
    else:
        print('invalid month input. must be "june" or "december"')
        return 0
    return cg
    
def Loads(pl, t):
    df_loads = pd.read_excel(r'ethylene 14.xlsx')
    for hour in range(t):
        pl += [df_loads.at[hour, 'P_L_s']]
    return pl

def Solar(ps, mon, t):
    if (mon=="june"):
        df_solar = pd.read_excel('pvwatts_hourly_Houston_20000kwh.xlsx', sheet_name='j')
        for hour in range(t):
            ps += [df_solar.at[hour, "ACMW"]] 
    elif (mon=="december"):
        df_solar = pd.read_excel('pvwatts_hourly_Houston_20000kwh.xlsx', sheet_name='d')
        for hour in range(t):
            ps += [df_solar.at[hour, "ACMW"]] 
    else:
        print('invalid month input. must be "june" or "december"')
        return 0
    return ps

def SellPrice_e(month, sp):
    if (month=="june"):
        df = pd.read_excel ('Mont Belvieu Ethylene Prices.xlsx', sheet_name='june')
        for n in range (0, 30):
            temp = [df.at[n, "$/ton"]] # cost per Mwh of solar
            for m in range (0, 24):
                sp += temp
    elif (month=="december"):
        df = pd.read_excel ('Mont Belvieu Ethylene Prices.xlsx', sheet_name='december')
        for n in range (0, 31):
            temp = [df.at[n, "$/ton"]] # cost per Mwh of solar
            for m in range (0, 24):
                sp += temp
    else:
        print("invalid month input")
        return 0
    return sp

def SellPrice_h(month, sp):
    if (month=="june"):
        for n in range (0, 30):
            for m in range (0, 24):
                sp += [1000]
    elif (month=="december"):
        for n in range (0, 31):
            for m in range (0, 24):
                sp += [1000]
    else:
        print("invalid month input")
        return 0
    return sp

def BuyPrice(month, bp):
    if (month=="june"):
        df = pd.read_excel ('Mont Belvieu Ethane Prices CV.xlsx', sheet_name='june')
        for n in range (0, 30):
            temp = [df.at[n, "$/ton"]] # cost per Mwh of solar
            for m in range (0, 24):
                bp += temp
    elif (month=="december"):
        df = pd.read_excel ('Mont Belvieu Ethane Prices CV.xlsx', sheet_name='december')
        for n in range (0, 31):
            temp = [df.at[n, "$/ton"]] # cost per Mwh of solar
            for m in range (0, 24):
                bp += temp
    else:
        print("invalid month input")
        return 0
    return bp

In [4]:
start = timeit.default_timer()
 
P_ramp = 20*60 # ramp values # ~20 MW/min for gas, coal would be ~2 MW/min # [MW/hr]
eff = 0.9 # battery efficiency
t = 720 # one month duration # [hours]
month = "june" # or "december"
cost = "wholesale" # "fixed" 
c_fix = 19.75 # june 19.75, december 21.19 # [$/MWh]
storage_limit_e = 250 # [mt]
storage_limit_h = 250 # [mt]
storage_limit_f = 750  # [mt]
outage = "no" # or "yes"
op_c = 85.75 # operational cost # [$]
pc_multiplier = 1
# outage_duration = 4 # [hours] # if outage = yes
# outage_start = 100 # [hour] # if outage = yes
battery_duration = 1 # 1 hour battery, 4 hour battery, etc

E_max_stor = []
prob_stor = []
C_b_stor = []
S_max_stor = []
P_c_stor = []
S_l_e_stor = []
S_l_h_stor = []
Pd_e_stor = []
C_g_stor = []
C_s_stor = []
St_e_stor = []
St_f_stor = []
N = 1 # horizon length # do 1, 4, 24
E_b_i = 0
St_e_i = 0
St_h_i = 0
St_f_i = 0

P_L = [] # loads # MW
P_L = Loads(P_L, t)
P_s = [] # power generated by solar # MW
P_s = Solar(P_s, month, t)
S_p_e = [] # sell price # [$/mt]
S_p_e = SellPrice_e(month, S_p_e) 
S_p_h = [] # sell price # [$/mt]
S_p_h = SellPrice_h(month, S_p_h)
B_p = [] # buy price # [$/mt]
B_p = BuyPrice(month, B_p)
C_gr = [] # cost of power from the grid #[$/mt]
C_gr = GridCost(C_gr, month, t, cost, c_fix)

# plt.plot(P_L, label="P_L")
# plt.plot(P_s, label="P_s")
# plt.plot(S_p_e, label="se")
# plt.plot(S_p_h, label="sh")
# plt.plot(B_p, label="bp")
# plt.plot(C_gr, label="cg")
# plt.legend()
# plt.show()

assert 0 < eff <= 1.0, "efficiency must be between (0,1]"

for m in range(t-N): 
    sta = timeit.default_timer()
    
    E_max = cp.Variable() # establishing battery capacity (MW)
    E_b = cp.Variable(N+1) # defining battery charge variable
    P_g = cp.Variable(N) # power drawn from the grid
    P_ch = cp.Variable(N) # battery charge
    P_dis = cp.Variable(N) # battery discharge []
    S_max = cp.Variable() # size of the solar panels 
    St_e = cp.Variable(N+1) # ethylene storage [mt]
    Sl_e = cp.Variable(N) # sold ethylene [mt]
    St_h = cp.Variable(N+1) # hydrogen storage [mt]
    Sl_h = cp.Variable(N) # sold hydrogen [mt]
    St_f = cp.Variable(N+1) # feedstock storage [mt]
    Bt_f = cp.Variable(N+1) # bought feedstock [mt]
    Pd_e = cp.Variable(N) # production of ethylene [mt]
    Pd_h = cp.Variable(N) # production of hydrogen [mt]
    Pd_f = cp.Variable(N) # production of feedstock [mt]

    C_s = S_max*Cost(1000000*0.89*20) # [$]
    C_b = E_max*(Cost(143000)) # [$]
    C_g = []
    for n in range(N):
        C_g += [(C_gr[m+n])*P_g[n]]  
    
    constraints = []   
    constraints += [P_g >= 0] # power draw is not negative # [MW >= MW]
    constraints += [P_L[m:N+m] == P_g + P_dis + -P_ch + P_s[m:N+m]*S_max] # the sources sum to load requirements # [MW = MW + MW + MW + MW]
    constraints += [E_b[0] == E_b_i] # battery charge at start is zero # [MW = MW]
    constraints += [P_ch >= 0] # charge into the is not negative # [MW >= MW]
    constraints += [P_dis >= 0] # discharge from battery is not negative # [MW >= MW]
    constraints += [S_max >= 0] # can't have negative solar capacity 
    constraints += [S_max <= 1] # solar 20000 kW or less
    constraints += [P_dis >= 0] # discharge from battery is not negative # [MW >= MW]
#     for n in range (0, N-1):     # ramp value constraints - power change between time steps can't exceed ramp limit
#         constraints += [(P_g[n] - P_g[n+1]) <= P_ramp] # [MW - MW <= MW]
#         constraints += [(P_g[n] - P_g[n+1]) >= -P_ramp]  # [MW - MW >= MW]
    constraints += [St_e[0] == St_e_i] # storage at time 0 = 0 # [mt = mt]
    constraints += [St_h[0] == St_h_i] # storage at time 0 = 0 # [mt = mt]
    constraints += [St_f[0] == St_f_i] # storage empty at start # [mt = mt]
    for n in range (0, N):
        constraints += [E_max/battery_duration >= P_ch[n]] # battery charge in one time step cannot be greater than the battery capacity #[MW=MW]
        constraints += [E_max/battery_duration >= P_dis[n]] # battery discharge in one time step cannot be greater than the battery capaticy #[MW=MW]
        constraints += [St_e[n] >= 0] # storage between 0 and limit # [mt = mt]
        constraints += [storage_limit_e >= St_e[n]] # storage between 0 and limit # [mt = mt]
        constraints += [St_h[n] >= 0] # storage between 0 and limit # [mt = mt]
        constraints += [storage_limit_h >= St_h[n]] # storage between 0 and limit # [mt = mt]
        constraints += [St_f[n] >= 0] # storage between 0 and limit # [mt = mt]
        constraints += [storage_limit_f >= St_f[n]] # storage between 0 and limit # [mt = mt]
        constraints += [Sl_e[n] >= 0] # selling ethylene not buying # [mt = mt]
        constraints += [Sl_e <= St_e[n]+Pd_e[n]] # sold ethylene less than produced # [mt = mt]
        constraints += [storage_limit_e >= Sl_e[n]] # can't sell more than storage limit in a given timestep # [mt = mt]
        constraints += [Sl_h[n] >= 0] # selling ethylene not buying # [mt = mt]
        constraints += [Pd_h[n] == Sl_h[n]] # can't sell more than storage limit in a given timestep # [mt = mt]
    #     constraints += [storage_limit_h >= Sl_h[n]] # can't sell more than storage limit in a given timestep # [mt = mt]
        constraints += [Bt_f[n] >= 0] # buying not selling # [mt = mt]
        constraints += [storage_limit_f >= Bt_f[n]] # can't buy more than storage limit in a given timestep # [mt = mt]
    for n in range (1, N+1):
        constraints += [E_b[n] == E_b[n-1] + eff*P_ch[n-1] - 1/eff*P_dis[n-1]] # battery effectiveness impacts charge/discharge amounts of battery # [Mw = MW + MW - MW]
        constraints += [St_e[n] == St_e[n-1] + Pd_e[n-1] - Sl_e[n-1]] # storage at next time = current + produced - sold # [mt = mt + mt - mt]
        constraints += [St_h[n] == 0] # storage at next time = current + produced - sold # [mt = mt + mt - mt]
    #     constraints += [St_h[n] == St_h[n-1] + Pd_h[n-1] - Sl_h[n-1]] # storage at next time = current + produced - sold # [mt = mt + mt - mt]
        constraints += [St_f[n] == St_f[n-1] + Bt_f[n-1] - Pd_f[n-1]] # storage at next time = current + bought - used # [mt = mt + mt - mt]
    for n in range (0, N+1):
        constraints += [E_b[n] >= 0] # battery charge is not negative # [MW >= MW]
        constraints += [E_max >= E_b[n]] # battery charge cannot be greater than the capacity  # [MW >= MW]
    if (outage=="yes"):
        for n in range (outage_start, outage_start + outage_duration):
            constraints += [P_g[n] == 0] # [MW = MW]   
            constraints += [Pd_e[n] >= 0.9*(len(P_L)/sum(P_L))*134.7*P_L[n]] # [mt >= 1/MW*mt*MW]
            constraints += [Pd_e[n] <= (len(P_L)/sum(P_L))*134.7*P_L[n]] # [mt <= 1/MW*mt*MW]
        for n in range(0, outage_start):
            constraints += [Pd_e[n] == (len(P_L)/sum(P_L))*134.7*P_L[n]]  # [mt == 1/MW*mt*MW]
        for n in range(outage_start + outage_duration, t):
            constraints += [Pd_e[n] == (len(P_L)/sum(P_L))*134.7*P_L[n]]  # [mt == 1/MW*mt*MW]
    elif (outage=="no"):
        for n in range(0, N):
            constraints += [Pd_e[n] == (len(P_L)/sum(P_L))*134.7*P_L[n]]  # [mt == 1/MW*mt*MW]
    for n in range(0, N):
        constraints += [Pd_h[n] == Pd_e[n] * 2.016/28.05] # hydrogen mol/ethylene mol conversion # [mt = mt * mol/mol]
        constraints += [Pd_f[n] == Pd_e[n] * 30.07/28.05] # ethane mol/ethylene mol conversion # [mt = mt * mol/mol]
    P_c = [] # bought ethylene including price # [$ = $/mt * mt]
    S_l_e = [] # sold ethylene including price # [$ = $/mt * mt]
    S_l_h = [] # sold ethylene including price # [$ = $/mt * mt]
    for n in range(N):
        P_c += [Bt_f[n] * B_p[n+m]]
        S_l_e += [Sl_e[n] * S_p_e[n+m]]
        S_l_h += [Sl_h[n] * S_p_h[n+m]]
        
    objective = op_c*sum(Pd_e) + sum(C_g[m:N+m]) + C_b + C_s + sum(P_c)*pc_multiplier - sum(S_l_e) - sum(S_l_h) # [$ = $/mt * mt + $ + $ + $ + $ - $ - $]

    prob = cp.Problem(cp.Minimize(objective), constraints)
    prob.solve() 
    print(prob.status)
    
    E_max_stor += [E_max.value]
    prob_stor += [prob.value]
    C_b_stor += [C_b.value]
    S_max_stor += [S_max.value]
    P_c_stor += [P_c[0].value]
    S_l_e_stor += [S_l_e[0].value]
    S_l_h_stor += [S_l_h[0].value]
    E_b_i = E_b[1].value
    St_e_i = St_e[1].value
    St_h_i = St_h[1].value
    St_f_i = St_f[1].value
    Pd_e_stor += [Pd_e[0].value]
    C_g_stor += [C_g[0].value]
    C_s_stor += [C_s.value]
    St_e_stor += [St_e[0].value]
    St_f_stor += [St_f[0].value]
    
    sto = timeit.default_timer()
    print("Time (seconds):", sto-sta)
    
#     print("a:", E_max.value, "b:", E_b.value, "c:", P_g.value, "d:", P_ch.value, "e:", P_dis.value, "f:", S_max.value,  "g:", St_e.value, "h:", Sl_e.value, "i:", St_h.value, "j:", Sl_h.value, "k:", St_f.value, "l:", Bt_f.value, "m:", Pd_e.value, "n:", Pd_h.value, "o:", Pd_f.value)
    
    
#printing results
obj_tot = op_c*sum(Pd_e_stor) + sum(C_g_stor) + max(C_b_stor) + max(S_max_stor) + sum(P_c_stor)*pc_multiplier - sum(S_l_e_stor) - sum(S_l_h_stor) # [$ = $/mt * mt + $ + $ + $ + $ - $ - $]
print("Battery Capacity [MW]: ", max(E_max_stor))
print("Cost of Battery [$]: ", max(C_b_stor))
print("Solar Size [MW]: ", max(S_max_stor)*20)
print("Cost of Bought Ethane [tons]: ", sum(P_c_stor))
print("Cost of Sold Ethylene [tons]: ", sum(S_l_e_stor))
print("Cost of Sold Hydrogen [tons]: ", sum(S_l_h_stor))
print("Overall Profit", -obj_tot)
if 0 > obj_tot:
    print("Profit")
else:
    print("Loss")

stop = timeit.default_timer()

print('Time (seconds): ', stop - start) 

# # battery charge variable for plotting
# E_plot = []
# for n in range (0, t):
#     E_plot = E_plot + [E_b.value[n]]
    
# #plotting the results  
# SMALL_SIZE = 20
# MEDIUM_SIZE = 22
# BIGGER_SIZE = 24

# plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
# plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
# plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
# plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
# plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
# plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
# plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure tit
# matplotlib.rc('font', size=SMALL_SIZE)
# matplotlib.rc('axes', titlesize=SMALL_SIZE)
# matplotlib.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
# matplotlib.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
# matplotlib.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
# matplotlib.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
# matplotlib.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure tit

# plt.plot(range(0,t), P_g.value, "red", label = 'Power from Grid') 
# plt.plot(range(0,t), P_dis.value, "blue", label = 'Power from Battery')
# plt.plot(range(0,t), P_L, "black", label = 'Demand')
# plt.plot(range(0,t), P_s*(S_max.value), "green", label = 'Power from Solar') 
# plt.plot(range(0,t), E_plot, "orange", label = 'Battery State of Charge')
# plt.xlabel("Time (hours)")
# plt.ylabel("Power Use (MW)")
# # plt.title("Normal Operation: June")
# leg = plt.legend()
# fig = plt.gcf()
# fig.set_size_inches(17, 11)

optimal
Time (seconds): 0.022292542000002413
infeasible
Time (seconds): 0.022539207999997757


SolverError: Solver 'ECOS' failed. Try another solver, or solve with verbose=True for more information.

In [ ]:
S_l_e_j = S_l_e_stor
S_l_h_j = S_l_h_stor
P_c_j = P_c_stor
C_s_j = C_s_stor
C_b_j = C_b_stor
C_g_j = C_g_stor
Pd_e_j = Pd_e_stor
St_e_j = St_e_stor
St_f_j = St_f_stor

In [ ]:
# S_l_e_d = S_l_e_stor
# S_l_h_d = S_l_h_stor
# P_c_d = P_c_stor
# C_s_d = C_s_stor
# C_b_d = C_b_stor
# C_g_d = C_g_stor
# Pd_e_d = Pd_e_stor
# St_e_d = St_e_stor
# St_f_d = St_f_stor

# Plotting

In [ ]:
SMALL_SIZE = 20
MEDIUM_SIZE = 22
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure tit
matplotlib.rc('font', size=SMALL_SIZE)
matplotlib.rc('axes', titlesize=SMALL_SIZE)
matplotlib.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
matplotlib.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
matplotlib.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
matplotlib.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
matplotlib.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure tit

In [ ]:
fig, ax = plt.subplots(figsize=(17,11))
width = 0.5
    
ax.bar(6, sum(S_l_e_j), width, color="purple", align='edge', label="sell ethylene june")
ax.bar(6, sum(S_l_h_j), width, color="blue", align='edge', bottom=sum(S_l_e_j), label="sell hydrogen june")
ax.bar(6.5, sum(P_c_j)*pc_multiplier, width, color="green", align='edge', label="bought ethane june")
ax.bar(6.5, max(C_s_j), width, color="gold", align='edge', bottom=sum(P_c_j)*pc_multiplier, label="solar cost june")
ax.bar(6.5, max(C_b_j), width, color="orange", align='edge', bottom=sum(P_c_j)*pc_multiplier+C_s_j, label="battery cost june")
ax.bar(6.5, sum(C_g_j), width, color="red", align='edge', bottom=sum(P_c_j)*pc_multiplier+C_s_j+C_b_j, label="grid cost june")
ax.bar(6.5, op_c*sum(Pd_e_j), width, color="darkred", align='edge', bottom=sum(P_c_j)*pc_multiplier+C_s_j+C_b_j+sum(C_g_j), label="operational cost june")

# ax.bar(12, sum(S_l_e_d), width, color="purple", hatch='/', align='edge', label="sell ethylene december")
# ax.bar(12, sum(S_l_h_d), width, color="blue", hatch='/', align='edge', bottom=sum(S_l_e_d), label="sell hydrogen december")
# ax.bar(12.5, sum(P_c_d)*pc_multiplier, width, color="green", hatch='/', align='edge', label="bought ethane december")
# ax.bar(12.5, max(C_s_d), width, color="gold", hatch='/', align='edge', bottom=sum(P_c_d)*pc_multiplier, label="solar cost december")
# ax.bar(12.5, max(C_b_d), width, color="orange", hatch='/', align='edge', bottom=sum(P_c_d)*pc_multiplier+C_s_d, label="battery cost december")
# ax.bar(12.5, sum(C_g_d), width, color="red", hatch='/', align='edge', bottom=sum(P_c_d)*pc_multiplier+C_s_d+C_b_d, label="grid cost december")
# ax.bar(12.5, op_c*sum(Pd_e_d), width, color="darkred", hatch='/', align='edge', bottom=sum(P_c_d)*pc_multiplier+C_s_d+C_b_d+sum(C_g_d), label="operational cost december")
ax.set_ylabel('Cost [$]')
ax.set_xlabel('Month')
ax.set_title('Monthly Costs and Profits')
ax.legend(loc="upper center")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(17,11))
ax2 = ax.twinx()
ax.set_xlabel('Time (Hour)')
ax.set_ylabel('Storage [mt]')
ax2.set_ylabel('Price [$]')
ax.plot(St_e_j, color="purple", label="Stored Ethylene")
ax.plot(St_f_j, color="green", label="Stored Ethane")
# ax.plot(St_e_d, color="purple", label="Stored Ethylene")
# ax.plot(St_f_d, color="green", label="Stored Ethane")
ax2.plot(S_p_e, color="orchid", label="Sell Price Ethylene")
ax2.plot(B_p, color="mediumaquamarine", label="Buy Price Ethane")
ax.legend(['Stored Ethylene', 'Stored Ethane'], loc='upper left')
ax2.legend(['Sell Price Ethylene', 'Buy Price Ethane'], loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
# todo:
# consider case without storage cost and with storage cost (use 1 hr, 4 hr, and 24 hr horizons (1 hr is no storage)) 